In [0]:
pip install lightgbm

In [0]:
pip install hyperopt

In [0]:
%pip install databricks-feature_engineering

In [0]:
%restart_python

In [0]:
import pandas as pd
import lightgbm as lgb
import pickle
import joblib
import os
from datetime import datetime, date 
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.lightgbm
from sklearn.metrics import classification_report,roc_auc_score,f1_score
import numpy as np
from mlflow.models.signature import infer_signature
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
import numpy as np
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll import scope
from databricks.feature_engineering import FeatureEngineeringClient
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup
from sklearn.preprocessing import LabelEncoder
import json
from mlflow.tracking import MlflowClient
import requests
from sklearn.metrics import accuracy_score
from mlflow.tracking import MlflowClient

In [0]:
dbutils.widgets.text("training_csv", "")

In [0]:
input_path = dbutils.widgets.get("training_csv")

In [0]:
print(input_path)

In [0]:
df= pd.read_csv(input_path)
df.shape

In [0]:
# Save train data
df[df['type'] == 'training'].to_csv("/Volumes/ispl_databricks/default/training/ff_bd/train.csv", index=False)

# Save validation data
df[df['type'] == 'validation'].to_csv("/Volumes/ispl_databricks/default/training/ff_bd/validation.csv", index=False)

In [0]:
df.dtypes.reset_index().to_csv("/Volumes/ispl_databricks/default/training/ff_bd/columns_datatypes.csv",index=False)

In [0]:
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype('category')

In [0]:
df['status_date'] = pd.to_datetime(df['status_date'], errors='coerce')
df['month_year'] = df['status_date'].dt.strftime('%m-%Y')

In [0]:
def age(born): 
    try:
        born = datetime.strptime(born, "%d-%m-%Y").date() 
        today = date.today() 
        age = today.year - born.year - ((today.month,  
                                        today.day) < (born.month,  
                                                        born.day))
        if age>=18:
            return age
        else:
            return (-1) 
    except:
        return -1
    
df['bue_age'] =df['bue_dob'].apply(age)

In [0]:
df['bue_perc_no_of_open_loans']=round(df['bue_no_of_open_loans']/df['bue_no_of_loans'],2)
df['bue_perc_no_of_open_loans_lst_6months']=round(df['bue_no_of_open_loans_lst_6months']/df['bue_no_of_loans'],2)
df['bue_perc_no_of_cc_loans']=round(df['bue_no_of_cc_loans']/df['bue_no_of_loans'],2)
df['bue_perc_no_of_cc_open_loans']=round(df['bue_no_of_cc_open_loans']/df['bue_no_of_open_loans'],2)
df['bue_perc_no_of_auto_open_loans']=round(df['bue_no_of_auto_open_loans']/df['bue_no_of_open_loans'],2)
df['bue_perc_no_of_consumer_loans']=round(df['bue_no_of_consumer_loans']/df['bue_no_of_loans'],2)
df['bue_perc_no_of_consumer_open_loans']=round(df['bue_no_of_consumer_open_loans']/df['bue_no_of_open_loans'],2)
df['bue_perc_no_of_personal_loans']=round(df['bue_no_of_personal_loans']/df['bue_no_of_loans'],2)
df['bue_perc_no_of_gold_loans']=round(df['bue_no_of_gold_loans']/df['bue_no_of_loans'],2)

In [0]:
df_num = df.select_dtypes(include=['float64','int64','int32'])
df_num.fillna(0,inplace=True)

In [0]:
df_cat = df.select_dtypes(include=['object','boolean','category'])

In [0]:
base_df2 = pd.concat([df_num,df_cat],axis=1)
base_df2.shape

In [0]:
null_percentages = base_df2.isnull().mean() * 100
columns_to_drop = null_percentages[null_percentages >= 40].index
base_df2.drop(columns=columns_to_drop, inplace=True)
len(columns_to_drop)

In [0]:
base_4=base_df2.drop(columns={'bue_dob','bue_name','imsi'})

In [0]:
# Define a dictionary for column-specific allowed categories
allowed_categories = {
    'currentNetworkRegion': [
        'Andhra Pradesh', 'Karnataka', 'Tamil Nadu', 'Maharashtra', 'Gujarat',
        'Delhi', 'Mumbai', 'Kerala', 'Bihar and Jharkhand', 'Rajasthan',
        'Madhya Pradesh', 'UP-East', 'UP-West', 'Chennai', 'Orissa',
        'West Bengal', 'Kolkata', 'Punjab', 'Assam', 'Haryana',
        'North East', 'Jammu & Kashmir', 'Himachal Pradesh'
    ],
    'email_emailFootprintStrength': [
        'Low', 'Very Low', 'Medium', 'High', 'Very High'
    ],
    'bue_score_comments': [
        'J-High Risk', 'I-Medium Risk', 'K-High Risk', 'H-Medium Risk',
        'M-Very High Risk', 'G-Low Risk', 'F-Low Risk', 'L-Very High Risk',
        'E-Low Risk', 'D-Very Low Risk', 'C-Very Low Risk',
        'Not Scored: Sufficient History Not Available',
        'Not Scored: Not Enough Info available on the customer',
        'Not Scored: No Activity seen on the customer (Inactive)',
        'B-Very Low Risk', 'A-Very Low Risk',
        'Not Scored: Cannot be classified',
        'Not Scored: No Updates available in last 36 months',
        'Not Scored: More than 50 active Accounts found',
        'Not Scored: Only a Guarantor'
    ],
    'phone_phoneFootprintStrengthOverall': [
        'High', 'Medium', 'Low', 'Very High', 'Very Low'
    ],
    'whatsapp': ['Account Found', 'Account Not Found', 'Error'],
    'numberBillingType': ['prepaid', 'postpaid', 'unknown']
}

# Apply mapping for all specified columns with custom logic for whatsapp and numberBillingType
for col, allowed_vals in allowed_categories.items():
    if col in base_4.columns:
        if col == 'whatsapp':
            # For 'whatsapp', map unknown categories to 'Error'
            base_4[col] = base_4[col].apply(
                lambda x: x if x in allowed_vals else 'Error'
            )
        elif col == 'numberBillingType':
            # For 'numberBillingType', map unknown categories to 'unknown'
            base_4[col] = base_4[col].apply(
                lambda x: x if x in allowed_vals else 'unknown'
            )
        else:
            # General logic: map unknown categories to 'others'
            base_4[col] = base_4[col].apply(
                lambda x: x if x in allowed_vals else 'others'
            )


In [0]:
model_name = 'ispl_databricks.model_logs.final_bd_model'

In [0]:

MODEL_URI = f"models:/{model_name}@champion"


In [0]:

client = MlflowClient()

In [0]:
model_info = mlflow.models.get_model_info(MODEL_URI)

In [0]:
input_schema = model_info.signature.inputs

In [0]:
feature_names = [col.name for col in input_schema.inputs]
feature_names = feature_names + ["loan_id"]


In [0]:
base_4 = base_4[feature_names]

In [0]:
base_4.dtypes

In [0]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in base_4.columns:
    dtype = base_4[col].dtype

    if col == "loan_id" and dtype.name == 'object':
        continue
    elif col == "loan_id" and dtype.name == 'category':
        base_4[col] = base_4[col].astype(str)
        continue
        
    
    if dtype.name in ["category", "object"]:
        le = LabelEncoder()
        base_4[col] = le.fit_transform(
            base_4[col].astype(str)
        )
        label_encoders[col] = le

In [0]:
base_4.dtypes

In [0]:
base_4 = base_4.drop_duplicates(subset=["loan_id"])

In [0]:
fe = FeatureEngineeringClient()

In [0]:
fe.create_table(
    name = 'ispl_databricks.model_logs.bd_sample',
    primary_keys=['loan_id'],
    df = spark.createDataFrame(base_4),
    description="feature store for sample data test"
)

In [0]:
fe.write_table(
    name = 'ispl_databricks.model_logs.bd_sample',
    df = spark.createDataFrame(base_4),
    mode = "merge"
)